<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Parallelization-Lab" data-toc-modified-id="Parallelization-Lab-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Parallelization Lab</a></span><ul class="toc-item"><li><span><a href="#Step-1:-Use-the-requests-library-to-retrieve-the-content-from-the-URL-below." data-toc-modified-id="Step-1:-Use-the-requests-library-to-retrieve-the-content-from-the-URL-below.-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Step 1: Use the requests library to retrieve the content from the URL below.</a></span></li><li><span><a href="#Step-2:-Use-BeautifulSoup-to-extract-a-list-of-all-the-unique-links-on-the-page." data-toc-modified-id="Step-2:-Use-BeautifulSoup-to-extract-a-list-of-all-the-unique-links-on-the-page.-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.</a></span></li><li><span><a href="#Step-3:-Use-list-comprehensions-with-conditions-to-clean-the-link-list." data-toc-modified-id="Step-3:-Use-list-comprehensions-with-conditions-to-clean-the-link-list.-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Step 3: Use list comprehensions with conditions to clean the link list.</a></span></li><li><span><a href="#Step-4:-Use-the-os-library-to-create-a-folder-called-wikipedia-and-make-that-the-current-working-directory." data-toc-modified-id="Step-4:-Use-the-os-library-to-create-a-folder-called-wikipedia-and-make-that-the-current-working-directory.-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Step 4: Use the os library to create a folder called <em>wikipedia</em> and make that the current working directory.</a></span></li><li><span><a href="#Step-5:-Write-a-function-called-index_page-that-accepts-a-link-and-does-the-following." data-toc-modified-id="Step-5:-Write-a-function-called-index_page-that-accepts-a-link-and-does-the-following.-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Step 5: Write a function called index_page that accepts a link and does the following.</a></span></li><li><span><a href="#Step-6:-Sequentially-loop-through-the-list-of-links,-running-the-index_page-function-each-time." data-toc-modified-id="Step-6:-Sequentially-loop-through-the-list-of-links,-running-the-index_page-function-each-time.-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Step 6: Sequentially loop through the list of links, running the index_page function each time.</a></span></li><li><span><a href="#Step-7:-Perform-the-page-indexing-in-parallel-and-note-the-difference-in-performance." data-toc-modified-id="Step-7:-Perform-the-page-indexing-in-parallel-and-note-the-difference-in-performance.-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Step 7: Perform the page indexing in parallel and note the difference in performance.</a></span></li></ul></li></ul></div>

# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

https://en.wikipedia.org/wiki/Data_science

In [15]:
import requests as req

url = 'https://en.wikipedia.org/wiki/Data_science'

In [16]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By # By es para buscar por tag, clase, id...
import time
from bs4 import BeautifulSoup as bs
import multiprocessing as mp
import re

In [17]:
x = req.get(url).text

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [18]:
soup = bs(x,'html.parser')

In [19]:
y = soup.find_all('a',{"href" : re.compile(r"/.*")})

In [20]:
z = [i['href'] for i in y]
z[0:20]

['/wiki/Main_Page',
 '/wiki/Special:Search',
 '/w/index.php?title=Special:CreateAccount&returnto=Data+science',
 '/w/index.php?title=Special:CreateAccount&returnto=Data+science',
 '/w/index.php?title=Special:UserLogin&returnto=Data+science',
 '/wiki/Help:Introduction',
 '/wiki/Special:MyTalk',
 '/wiki/Special:MyContributions',
 '/wiki/Main_Page',
 '/wiki/Wikipedia:Contents',
 '/wiki/Portal:Current_events',
 '/wiki/Special:Random',
 '/wiki/Wikipedia:About',
 '//en.wikipedia.org/wiki/Wikipedia:Contact_us',
 'https://donate.wikimedia.org/wiki/Special:FundraiserRedirector?utm_source=donate&utm_medium=sidebar&utm_campaign=C13_en.wikipedia.org&uselang=en',
 '/wiki/Help:Contents',
 '/wiki/Help:Introduction',
 '/wiki/Wikipedia:Community_portal',
 '/wiki/Special:RecentChanges',
 '/wiki/Wikipedia:File_upload_wizard']

## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [21]:
domain = 'http://wikipedia.org'

In [22]:
rel = list(filter(lambda x : bool(re.search("^/", x)),z))
abs = list(filter(lambda x :  not bool(re.search("^/", x)),z))

In [23]:
abs += list(filter( lambda x : bool(re.search(".*.org", x)) , rel))

In [24]:
abs = list(filter( lambda x : not bool(re.search("//[a-z]{2}.w", x)), abs)) + \
     list(filter( lambda x : bool(re.search("//en.", x)), abs))

In [25]:
rel = list(filter(lambda x : not bool(re.search("%", x)),rel))
rel = [i for i in rel if i not in abs]
rel = list(map(lambda x : domain + x, rel))

In [35]:
clean = []
for i in rel+abs:
    if i not in clean:
        clean.append(i)

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [26]:
import os

In [27]:
path = os.path.join('','wikipedia')

In [14]:
os.mkdir(path)


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'wikipedia'

In [28]:
os.chdir('wikipedia/')

In [ ]:
os.getcwd()

'c:\\Users\\jorge\\Ironhack\\Labs\\4.4-lab-parallelization\\your-code\\wikipedia'

## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [29]:
from slugify import slugify
from urllib.request import urlopen
unicode=str()

In [30]:
def index_page(s):
    try:
        sl = slugify(s)
        data = urlopen(s).read().decode('utf-8').split('\n')
        with open(sl+'.html','w') as f:
            f.writelines(data)
    except:
        pass
    

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [ ]:
%%time

for i in map(index_page,clean):
    print('done',end = " ")
print()

done done done done done done done done done done done done done 

## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [31]:
path = os.path.join('','multi')


In [32]:
os.mkdir(path)
os.chdir(path)

In [34]:
pool = mp.Pool(3) 
z = map(index_page, abs + rel)
res=pool.map_async(lambda x : None, z)
pool.close()